<a href="https://colab.research.google.com/github/nandhukumar86/CapstonePneumoniaDetection/blob/main/DenseNet_TransferLearning_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline
import cv2
import skimage.io as io
import os
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D, Concatenate, Reshape, UpSampling2D
import pickle
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.losses import binary_crossentropy
import gc
from tensorflow.keras.applications import DenseNet121, DenseNet201
from tensorflow.keras.applications import VGG16
from google.colab.patches import cv2_imshow
from tensorflow import reduce_sum
from tensorflow.keras.backend import log,epsilon

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file = open('/content/drive/MyDrive/CapstoneDataSet/dataset/RezisedLabel-1.pickle', 'rb')
dfData = pickle.load(file)
file.close()

In [4]:
dfData_0 = dfData[dfData['Target'] == 0].sample(9555)
dfData_1 = dfData[dfData['Target'] == 1].sample(9555)

In [5]:
dfData = pd.concat([dfData_0, dfData_1], axis=0, ignore_index=True)

In [6]:
dfData = dfData.sample(frac=1)

In [ ]:
y_classifier = []
X = []
i = 0
for pId in dfData['patientId'].unique():
  data = dfData[dfData['patientId'] == pId][['Image256','xo','yo','widtho','heighto','Target']]
  image1c = np.array(data.iloc[0][0]/255,'float16')
  X.append(image1c)
  y_classifier.append(data.iloc[0][5])
  if (i%1000 == 0):
    print(i, " completed")
  i = i + 1

In [8]:
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X, y_classifier, test_size=0.30, random_state=100)

In [9]:
tensorflow.keras.backend.clear_session()

In [10]:
gc.collect()

36

In [11]:
model = DenseNet201(input_shape=(256,256,3),include_top=False,weights='imagenet')

In [12]:
lengthLayers = len(model.layers)
pcLayers = int(0.90 * lengthLayers)
for i in range(pcLayers):
  model.layers[i].trainable = False
  

In [13]:
layer8x8 = model.get_layer('conv5_block32_2_conv').output

yy = Flatten()(layer8x8)
yy = Dense(2, activation='softmax')(yy)

model_classifier = Model(model.input, yy)

In [14]:
gc.collect()

0

In [15]:
X_train_c = np.array(X_train_c)
y_train_c = np.array(y_train_c)
X_val_c = np.array(X_val_c)
y_val_c = np.array(y_val_c)

In [16]:
X_train_c.shape, y_train_c.shape, X_val_c.shape, y_val_c.shape

((10896, 256, 256), (10896,), (4671, 256, 256), (4671,))

In [17]:
X_train_c = np.array([X_train_c.T,X_train_c.T,X_train_c.T]).T
y_train_c = np.array([y_train_c, ~y_train_c+2]).T
X_val_c = np.array([X_val_c.T,X_val_c.T,X_val_c.T]).T
y_val_c = np.array([y_val_c, ~y_val_c+2]).T

In [18]:
X_train_c.shape, y_train_c.shape, X_val_c.shape, y_val_c.shape

((10896, 256, 256, 3), (10896, 2), (4671, 256, 256, 3), (4671, 2))

In [19]:
gc.collect()

382

In [20]:
model_classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

earlystop_classifier = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.01)

model_classifier.fit(X_train_c, y_train_c, 
                     epochs=10, batch_size=16, 
                     validation_data=(X_val_c, y_val_c), 
                     verbose=1, 
                     callbacks=[earlystop_classifier]) 


Epoch 1/10
681/681 [==============================] - 103s 116ms/step - loss: 0.4726 - accuracy: 0.7785 - val_loss: 0.4629 - val_accuracy: 0.7803
Epoch 2/10
681/681 [==============================] - 64s 94ms/step - loss: 0.4326 - accuracy: 0.7956 - val_loss: 0.4664 - val_accuracy: 0.7778
Epoch 3/10
681/681 [==============================] - 64s 94ms/step - loss: 0.3951 - accuracy: 0.8167 - val_loss: 0.5157 - val_accuracy: 0.7692


In [21]:
gc.collect()

2265

In [ ]:
! pip install tensorflowjs

In [23]:
import tensorflowjs
tensorflowjs.converters.save_keras_model(model_classifier,'/content/drive/MyDrive/CapstoneDataSet/classificationModel')

In [24]:
gc.collect()

120

In [47]:
a = 200
b = 220
y_test_predict = model_classifier.predict(X_val_c[a:b])
act = y_val_c[a:b]

In [48]:
for i in range(10):
  print(y_test_predict[i].argmax(), act[i].argmax())

0 0
1 1
1 1
0 0
1 1
0 0
0 0
1 1
0 0
0 0
